In [1]:
import logging
import os
import openai
import itertools
import inspect


from actionweaver.llms.azure.chat import ChatCompletion
from actionweaver.llms.azure.tokens import TokenUsageTracker
from actionweaver import action


def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')

In [2]:
logging.basicConfig(
    filename='agent.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [3]:
@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"

In [4]:
chat = ChatCompletion(
    model="gpt-35-turbo-0613-16k", 
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview", 
    token_usage_tracker = TokenUsageTracker(budget=2000, logger=logger), 
    logger=logger)

In [5]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [], stream=False))

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

In [9]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [get_current_time], stream=False))

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

In [8]:
print_output(chat.create([{"role": "user", "content": "what time is it"}], actions = [], stream=True))

I'm sorry, I am an AI language model and I do not have access to real-time information. Please check the time on your device or ask someone nearby for the current time.

In [9]:
print_output(chat.create([{"role": "user", "content": "what time is it, print in chinese"}], actions = [get_current_time], stream=True))

当前时间是2023年11月23日 19:52:22.840094。

In [10]:
from pydantic import BaseModel
from actionweaver.actions.factories.pydantic_model_to_action import action_from_model

class User(BaseModel):
    name: str
    age: int

action_from_model(User).invoke(chat, [{"role": "user", "content": "Tom is 31 years old"}])

User(name='Tom', age=31)

In [11]:
get_current_time.invoke(chat, [{"role": "user", "content": "what time is it now"}])


'The current time is 19:52 (7:52 PM).'

In [16]:
from actionweaver.mixins.examples import LangChainTools


class AgentV0(LangChainTools):
    def __init__(self, logger):
        self.llm = chat
        self.messages = [{"role": "system", "content": "You are a resourceful assistant."}]

    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        response = self.llm.create(messages=self.messages, actions = [self.search], stream=True)

        return response


agent = AgentV0(logger)

In [17]:
print_output(agent("search what is Langchain, what is haystack"))

Langchain is a framework for developing applications powered by language models. It simplifies the process of creating generative AI application interfaces and provides tools and APIs for developers to build context-aware, reasoning applications. Langchain is an open-source library that enables the development of applications using large language models (LLMs). It offers flexibility and an extensive toolkit for building LLM applications.

Haystack, on the other hand, is an open-source Python framework for building production-ready LLM applications. It provides tooling for every stage of the NLP project life cycle and is built around the concept of pipelines. Haystack offers components connected together to perform NLP tasks and orchestrate LLMs. It is designed to aggregate information in meaningful ways and is suitable for building applications that require access to and analysis of large amounts of text data.

Please note that there may be other unrelated results for the term "Haystac